# **Setup the Environment**

In [ ]:
!pip install nilearn==0.5.2 niwidgets==0.2.2 numpy-stl==2.11.2
!pip install open3d k3d
!pip3 install ipympl

**After succesful installation, restart and clear cell outputs.**
Comment the above lines, hit run all

# **Import all dependencies**

In [ ]:
import os, glob
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import load_model
from tensorflow import keras
import shutil, pathlib, fnmatch
import PIL
from niwidgets import NiftiWidget
import numpy as np
import open3d as o3d
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from skimage import measure
from stl import mesh
from sklearn.metrics import confusion_matrix

# **Pre-processing**

In [ ]:
# CONSTANTS!!!

# STEP 1 - Load and visualize data
dataInputPath = '../input/covid19-ct-scans'
imagePathInput = os.path.join(dataInputPath, 'ct_scans/')
maskPathInput = os.path.join(dataInputPath, 'infection_mask/')

dataOutputPath = './slices/'
imageSliceOutput = os.path.join(dataOutputPath, 'img/')
maskSliceOutput = os.path.join(dataOutputPath, 'mask/')
os.makedirs(imageSliceOutput,mode = 0o666)
os.makedirs(maskSliceOutput,mode = 0o666)
# STEP 2 - Image normalization
HOUNSFIELD_MIN = -1000
HOUNSFIELD_MAX = 2000
HOUNSFIELD_RANGE = HOUNSFIELD_MAX - HOUNSFIELD_MIN

# STEP 3 - Slicing and saving
SLICE_X = True
SLICE_Y = True
SLICE_Z = True

SLICE_DECIMATE_IDENTIFIER = 3

In [ ]:
# Load image and see max min Hounsfield units
imgPath = os.path.join(imagePathInput, "coronacases_org_002.nii")
img = nib.load(imgPath).get_fdata()
np.min(img), np.max(img), img.shape, type(img)

In [ ]:
# Load image mask and see max min Hounsfield units
maskPath = os.path.join(maskPathInput, "coronacases_002.nii")
mask = nib.load(maskPath).get_fdata()
np.min(mask), np.max(mask), mask.shape, type(mask)

In [ ]:
# Show image slice
imgSlice = mask[150,:,:]
plt.imshow(imgSlice, cmap='gray')
plt.show()

In [ ]:
# Normalize image
def normalizeImageIntensityRange(img):
    img[img < HOUNSFIELD_MIN] = HOUNSFIELD_MIN
    img[img > HOUNSFIELD_MAX] = HOUNSFIELD_MAX
    return (img - HOUNSFIELD_MIN) / HOUNSFIELD_RANGE

nImg = normalizeImageIntensityRange(img)
np.min(nImg), np.max(nImg), nImg.shape, type(nImg)

In [ ]:
# Read image or mask volume
def readImageVolume(imgPath, normalize=False):
    img = nib.load(imgPath).get_fdata()
    if normalize:
        return normalizeImageIntensityRange(img)
    else:
        return img
    
readImageVolume(imgPath, normalize=True)
readImageVolume(maskPath, normalize=False)

In [ ]:
# Save volume slice to file
def saveSlice(img, fname, path):
    print("image shape:",img.shape) 
    img = np.uint8(img * 255)
    fout = os.path.join(path, f'{fname}.png')
    cv2.imwrite(fout, img)
    print(f'[+] Slice saved: {fout}', end='\r')
    
# saveSlice(nImg[20,:,:], 'test', imageSliceOutput)
# saveSlice(mask[20,:,:], 'test', maskSliceOutput)

In [ ]:
# Slice image in all directions and save
def sliceAndSaveVolumeImage(vol, fname, path):
    (dimx, dimy, dimz) = vol.shape
    print(dimx, dimy, dimz)
    cnt = 0
    if SLICE_X:
        cnt += dimx
        print('Slicing X: ')
        for i in range(dimx):
            saveSlice(vol[i,:,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_x', path)
            
    if SLICE_Y:
        cnt += dimy
        print('Slicing Y: ')
        for i in range(dimy):
            saveSlice(vol[:,i,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_y', path)
            
    if SLICE_Z:
        cnt += dimz
        print('Slicing Z: ')
        for i in range(dimz):
            saveSlice(vol[:,:,i], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_z', path)
    return cnt

In [ ]:
# Read and process image volumes
for index, filename in enumerate(sorted(glob.iglob(imagePathInput+'coronacases*.nii'))):
    img = readImageVolume(filename, True)
    print(filename, img.shape, np.sum(img.shape), np.min(img), np.max(img))
    numOfSlices = sliceAndSaveVolumeImage(img, 'lungs'+str(index), imageSliceOutput)
    print(f'\n{filename}, {numOfSlices} slices created \n')

In [ ]:
# Read and process image mask volumes
for index, filename in enumerate(sorted(glob.iglob(maskPathInput+'coronacases*.nii'))):
    img = readImageVolume(filename, False)
    print(filename, img.shape, np.sum(img.shape), np.min(img), np.max(img))
    numOfSlices = sliceAndSaveVolumeImage(img, 'lungs'+str(index), maskSliceOutput)
    print(f'\n{filename}, {numOfSlices} slices created \n')

In [ ]:
count=0
for filename in os.listdir("./slices/mask"):
    count+=1
count*2

# **Lung Segmentation using U-net**

Organizing data in appropriate train & test folders

In [ ]:
dataPath = './'
traindataSliceOutput = os.path.join(dataPath, 'training/img/img/')
trainmaskSliceOutput = os.path.join(dataPath, 'training/mask/mask/')
testdataSliceOutput = os.path.join(dataPath, 'test/img/img/')
testmaskSliceOutput = os.path.join(dataPath, 'test/mask/mask/')


In [ ]:
#function to move data from source to destination using string regex
def move_dir(src: str, dst: str, pattern: str = '*'):
    if not os.path.isdir(dst):
        pathlib.Path(dst).mkdir(parents=True, exist_ok=True)
    for f in fnmatch.filter(os.listdir(src), pattern):
        shutil.move(os.path.join(src, f), os.path.join(dst, f))

In [ ]:
os.makedirs(traindataSliceOutput,mode = 0o666)
os.makedirs(trainmaskSliceOutput,mode = 0o666)
os.makedirs(testdataSliceOutput,mode = 0o666)
os.makedirs(testmaskSliceOutput,mode = 0o666)

Uncomment if more test samples are required. Currently using 2 test, 8 train.

In [ ]:
# move_dir(imageSliceOutput,testdataSliceOutput,'lungs6*.png')
# move_dir(imageSliceOutput,testdataSliceOutput,'lungs7*.png')
move_dir(imageSliceOutput,testdataSliceOutput,'lungs8*.png')
move_dir(imageSliceOutput,testdataSliceOutput,'lungs9*.png')
# move_dir(maskSliceOutput,testmaskSliceOutput,'lungs6*.png')
# move_dir(maskSliceOutput,testmaskSliceOutput,'lungs7*.png')
move_dir(maskSliceOutput,testmaskSliceOutput,'lungs8*.png')
move_dir(maskSliceOutput,testmaskSliceOutput,'lungs9*.png')

In [ ]:
#Move the remaining files into training data folder
move_dir(imageSliceOutput,traindataSliceOutput,'*.png')
move_dir(maskSliceOutput,trainmaskSliceOutput,'*.png')

In [ ]:
# Define constants
SEED = 909
BATCH_SIZE_TRAIN = 16
BATCH_SIZE_TEST = 16

IMAGE_HEIGHT = 512
IMAGE_WIDTH = 256
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)

data_dir = './'
data_dir_train = os.path.join(data_dir, 'training')
# The images should be stored under: "data/slices/training/img/img"
data_dir_train_image = os.path.join(data_dir_train, 'img')
# The images should be stored under: "data/slices/training/mask/img"
data_dir_train_mask = os.path.join(data_dir_train, 'mask')

data_dir_test = os.path.join(data_dir, 'test')
# The images should be stored under: "data/slices/test/img/img"
data_dir_test_image = os.path.join(data_dir_test, 'img')
# The images should be stored under: "data/slices/test/mask/img"
data_dir_test_mask = os.path.join(data_dir_test, 'mask')

NUM_TRAIN = len(os.listdir(traindataSliceOutput))
NUM_TEST = len(os.listdir(testdataSliceOutput))

NUM_OF_EPOCHS = 5

Folder structure organization done! Create Data generators for training.

In [ ]:
#Uncomment below lines, if Data augmentation during training is required.
def create_segmentation_generator_train(img_path, msk_path, BATCH_SIZE):
    data_gen_args = dict(rescale=1./255
#                      featurewise_center=True,
#                      featurewise_std_normalization=True,
#                      rotation_range=90
#                      width_shift_range=0.2,
#                      height_shift_range=0.2,
#                      zoom_range=0.3
                        )
    datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = datagen.flow_from_directory(img_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    msk_generator = datagen.flow_from_directory(msk_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    return zip(img_generator, msk_generator)

# Remember not to perform any image augmentation in the test generator!
def create_segmentation_generator_test(img_path, msk_path, BATCH_SIZE):
    data_gen_args = dict(rescale=1./255)
    datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = datagen.flow_from_directory(img_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    msk_generator = datagen.flow_from_directory(msk_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    return zip(img_generator, msk_generator)

In [ ]:
train_generator = create_segmentation_generator_train(data_dir_train_image, data_dir_train_mask, BATCH_SIZE_TRAIN)
test_generator = create_segmentation_generator_test(data_dir_test_image, data_dir_test_mask, BATCH_SIZE_TEST)

In [ ]:
#Function to compare between input, mask and prediction
def display(display_list):
    plt.figure(figsize=(15,15))
    
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]), cmap='gray')
    plt.show()

In [ ]:
#Function to display top n images in datagen
def show_dataset(datagen, num=1):
    for i in range(0,num):
        image,mask = next(datagen)
        display([image[0], mask[0]])

In [ ]:
#Visualize top 3 image comparisons in datagen
show_dataset(train_generator, 3)

U-net model development

In [ ]:
def unet(n_levels, initial_features=32, n_blocks=2, kernel_size=3, pooling_size=2, in_channels=1, out_channels=1):
    inputs = keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, in_channels))
    x = inputs
    
    convpars = dict(kernel_size=kernel_size, activation='relu', padding='same')
    
    #downstream
    skips = {}
    for level in range(n_levels):
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
        if level < n_levels - 1:
            skips[level] = x
            x = keras.layers.MaxPool2D(pooling_size)(x)
            
    # upstream
    for level in reversed(range(n_levels-1)):
        x = keras.layers.Conv2DTranspose(initial_features * 2 ** level, strides=pooling_size, **convpars)(x)
        x = keras.layers.Concatenate()([x, skips[level]])
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
            
    # output
    activation = 'sigmoid' if out_channels == 1 else 'softmax'
    x = keras.layers.Conv2D(out_channels, kernel_size=1, activation=activation, padding='same')(x)
    
    return keras.Model(inputs=[inputs], outputs=[x], name=f'UNET-L{n_levels}-F{initial_features}')
        

In [ ]:
EPOCH_STEP_TRAIN = NUM_TRAIN // BATCH_SIZE_TRAIN
EPOCH_STEP_TEST = NUM_TEST // BATCH_SIZE_TEST
#Instantiate U-net with 4 levels
model = unet(4)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=EPOCH_STEP_TRAIN, 
                    validation_data=test_generator, 
                    validation_steps=EPOCH_STEP_TEST,
                   epochs=NUM_OF_EPOCHS)

In [ ]:
model.save(f'UNET-LungSegmentation_{IMAGE_HEIGHT}_{IMAGE_WIDTH}.h5')

In [ ]:
test_generator = create_segmentation_generator_test(data_dir_test_image, data_dir_test_mask, 1)

In [ ]:
def show_prediction(datagen, num=1):
    for i in range(0,num):
        image,mask = next(datagen)
        pred_mask = model.predict(image)[0] > 0.5
        display([image[0], mask[0], pred_mask])

In [ ]:
show_prediction(test_generator, 3)